In [ ]:
import os
import re
import pandas as pd

def BuildDataframe(directory):
    data = {'image_path': [], 'label': []}
    pattern = r'diseased'

    for plant in os.listdir(directory):
        class_dir = os.path.join(directory, plant)

        if os.path.isdir(class_dir):
            match = re.search(pattern, plant)
            label = "healthy"
            if match:
                label = "diseased"
                    
            print("Extracting ", plant, "...")            
            for image_file in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_file)
                    
                data['image_path'].append(image_path)
                data['label'].append(label)

    print("Data correctly extracted.")
    dataframe = pd.DataFrame(data)
    return dataframe


In [ ]:
train_dir = '/kaggle/input/plant-leaves-for-image-classification/Plants_2/train'
train_dataframe = BuildDataframe(train_dir)

test_dir = '/kaggle/input/plant-leaves-for-image-classification/Plants_2/test'
test_dataframe = BuildDataframe(test_dir)

valid_dir = '/kaggle/input/plant-leaves-for-image-classification/Plants_2/valid'
valid_dataframe = BuildDataframe(valid_dir)

sample_df = train_dataframe.sample(n=10)
print(sample_df)


In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU config OK")
else:
    print("Error using GPU, CPU will be used")
tf.keras.backend.clear_session()

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
input_shape = (224, 224, 3)
num_classes = 2
batch_size = 32
epochs = 10


## Data generator



In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)


# Loading the data

In [ ]:
print(train_dataframe)

In [ ]:
train_directory = '/kaggle/input/plant-leaves-for-image-classification/Plants_2/train'
test_directory = '/kaggle/input/plant-leaves-for-image-classification/Plants_2/test'
valid_directory = '/kaggle/input/plant-leaves-for-image-classification/Plants_2/valid'

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_dataframe,
    directory=train_directory,
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_dataframe,
    directory=test_directory,
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = test_datagen.flow_from_dataframe(
    dataframe=valid_dataframe,
    directory=valid_directory,
    x_col='image_path',
    y_col='label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)


# Loading the model

In [ ]:
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

# Bloquer la partie convolutionnelle du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Ajouter les nouvelles couches pour la classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Construction du modèle final
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=epochs, validation_data=valid_generator)

# Test the model

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)


In [ ]:
import tensorflow as tf
from tensorflow import keras

model.save('Plant_health_model.h5')
print("Model saved successfully.")